In [35]:
import random
from datetime import datetime
import pandas as pd

In [36]:
def random_roll():
    last_time = datetime.now().timestamp()
    while True:
        time_now = datetime.now().timestamp()
        if last_time != time_now:
            random.seed(time_now)
            last_time = time_now
        else:
            random.seed(random.random())
        yield random.random()
rnd = random_roll()

class Model:
    def __init__(self, config):
        self.config = config # вхідні параметри зберігаємо
        self.profit = config.profit # прибуток витягуємо, але можна і писати поверх вхідного значення
        self.store = []
        self.attributes = ["day","start_stock","number","demand","ordered","replenished","end_stock", "profit"]
        self.deficit_days = 0
        self.day = 0
        self.end_stock = config.start_stock # треба пам'ятати, який запас був учора ввечорі
        self.order_days = 0 # лічільник того, скільки днів щоби до запасу надійшло поповнення після замовлення
        # перетворимо зважені ймовірності на коефіцієнти (30 -> 0.3)
        probabilities = []
        w_sum = sum(config.weights)
        for w in config.weights:
            probabilities.append(w/w_sum)
        # додамо акумульовані ймовірності, що вони дорівнюють сумі усіх попередніх ймовірностей
        p_accumulator = 0
        self.accumulation = []
        for p in probabilities:
            p_accumulator += p
            self.accumulation.append(p_accumulator)
    def time_increment(self):
        # дії на початку дня
        self.day += 1
        start_stock = self.end_stock
        replenished = 0
        ordered = 0
        # Надходження замовлення
        if self.order_days > 0:
            self.order_days -= 1
            if self.order_days == 0:
                replenished = self.config.replenish_quantity
                start_stock += self.config.replenish_quantity
        else: # Замовлення немає
            # треба розмістити замовлення, якщо запас просів нижче порогу 
            if start_stock <= self.config.replenish_condition:
                self.order_days = self.config.cycle_days
                ordered = self.config.replenish_quantity
                # відраховуємо вартість замовлення з загального бюджету
                self.profit -= self.config.replenish_expense + self.config.replenish_price * self.config.replenish_quantity
        number = next(rnd)
        demand = 0
        index = 0
        for n in self.accumulation:
            if n <= number:
                demand = self.config.quantities[index]
            index += 1
        self.end_stock = start_stock - demand
        if self.end_stock < 0:
            self.deficit_days += 1
        # нараховуємо прибуток відповідно попиту
        self.profit += demand * self.config.demand_price
        return [
            self.day,
            start_stock,
            number,
            demand,
            ordered,
            replenished,
            self.end_stock,
            self.profit
        ]
    def model(self, timespan = 10):
        for _ in range(timespan):
            self.store.append(self.time_increment())
        df = pd.DataFrame(self.store, columns=self.attributes)
        print(df)
        return(df)

class Object:
    def __init__(self, **attributes):
        self.__dict__.update(attributes)

In [41]:
demand_config = Object(
    start_stock = 10,
    weights = [10, 30, 30, 20, 5, 5],
    quantities = [1, 2, 3, 4, 5, 6],
    cycle_days = 2,
    replenish_quantity = 10,
    replenish_condition = 8,
    replenish_price = 5500,
    replenish_expense = 1400,
    demand_price = 9000,
    profit = -5000
)

timespan = 3650
model = Model(demand_config)
df = model.model(timespan)

       day  start_stock    number  demand  ordered  replenished  end_stock  \
0        1           10  0.464389       2        0            0          8   
1        2            8  0.880168       3       10            0          5   
2        3            5  0.430080       2        0            0          3   
3        4           13  0.689376       2        0           10         11   
4        5           11  0.858479       3        0            0          8   
...    ...          ...       ...     ...      ...          ...        ...   
3645  3646            6  0.465640       2       10            0          4   
3646  3647            4  0.083080       0        0            0          4   
3647  3648           14  0.221446       1        0           10         13   
3648  3649           13  0.816303       3        0            0         10   
3649  3650           10  0.848308       3        0            0          7   

        profit  
0        13000  
1       -16400  
2         16

In [42]:
print(f"{model.deficit_days} deficit days out of {timespan} days modeled")
print(f"with {model.deficit_days/timespan*100}% of deficit probability")
print(f"and {model.profit} conventional units of profit")

51 deficit days out of 3650 days modeled
with 1.3972602739726028% of deficit probability
and 24482800 conventional units of profit


In [43]:
demand_config = Object(
    start_stock = 10,
    weights = [10, 30, 30, 20, 5, 5],
    quantities = [1, 2, 3, 4, 5, 6],
    cycle_days = 2,
    replenish_quantity = 20,
    replenish_condition = 10,
    replenish_price = 5500,
    replenish_expense = 1400,
    demand_price = 9000,
    profit = -5000
)

timespan = 3650
model = Model(demand_config)
df = model.model(timespan)

       day  start_stock    number  demand  ordered  replenished  end_stock  \
0        1           10  0.849307       3       20            0          7   
1        2            7  0.443606       2        0            0          5   
2        3           25  0.730833       3        0           20         22   
3        4           22  0.572578       2        0            0         20   
4        5           20  0.577726       2        0            0         18   
...    ...          ...       ...     ...      ...          ...        ...   
3645  3646           11  0.762005       3        0            0          8   
3646  3647            8  0.508499       2       20            0          6   
3647  3648            6  0.128213       1        0            0          5   
3648  3649           25  0.022162       0        0           20         25   
3649  3650           25  0.680553       2        0            0         23   

        profit  
0       -89400  
1       -71400  
2       -444

In [40]:
print(f"{model.deficit_days} deficit days out of {timespan} days modeled")
print(f"with {model.deficit_days/timespan*100}% of deficit probability")
print(f"and {model.profit} conventional units of profit")

1 deficit days out of 3650 days modeled
with 0.0273972602739726% of deficit probability
and 24857400 conventional units of profit
